In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [4]:
from utils.character_attributes_extraction import character_names_from_text, character_attributes_from_text, character_active_verbs_from_text, character_patient_verbs_from_text

/home/cat/EPFL/Fall 2023/ADA/project_repo/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
plots =  pd.read_csv(
    'data/MovieSummaries/plot_summaries.txt', 
    sep='\t', 
    names=['wiki_id', 'plot']
)

plots['plot'] = plots['plot'].apply(lambda x: ' '.join(x.split()))

plots.head(5)

,wiki_id,plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six year...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


# Extract characters names

## Spacy

[Spacy](https://spacy.io/usage/linguistic-features) is a really nice library that allows to do many nlp tasks easilly.
We will use named entity recognition and dependency parsing the most. Also we will probably use built-in word2vec.

In [6]:
import spacy

nlp = spacy.load("en_core_web_md")

In [7]:
doc = nlp("As Gregor Samsa awoke one morning from uneasy dreams he found himself transformed in his bed into an enormous insect.")

In [8]:
from spacy import displacy
displacy.render(doc, style='dep')

In [9]:
plot = plots.loc[51]['plot']
print(plot)

A farmer is busy with hoeing while his son Porky is ploughing the fields with his horse Dobbin. Hank Horsefly speeds up the process. The farmer and Porky are about to take a turn for the worst as Mr. Viper The Snake comes with a Mortgage form ready to evict them unless a sum of rent money is paid. Porky applies for a job as horse driving milkman with a strict condition not to break a single bottle. Porky is doing well until Hank having followed their trail, sends Dobbin going at full speed and crashing, causing all the milk bottles to break. As Porky despairs, Dobbin accidentally enters a horse race. When the race starts, Dobbin isn't getting far, until Hank kick starts Dobbin to overtake every racer and wins a $40,000 prize. Porky makes it to the farm in the nick of time, riding in a roofless limo. Porky pays the owed money to Mr. Viper and Hank gives him a kick.


In [10]:
character_attributes_from_text(plot)

{'Porky': ['son'], 'Dobbin': ['horse']}

In [11]:
character_active_verbs_from_text(plot)

{'Hank Horsefly': ['speeds', 'followed'],
 'Viper The Snake': ['comes'],
 'Porky': ['applies', 'doing', 'makes', 'pays'],
 'Dobbin': ['going', 'enters', 'getting']}

In [12]:
%%script false --no-raise-error

character_list = []

for index, row in tqdm([row for row in plots.iterrows()]):
    plot = row['plot'] 
    character_names = character_names_from_text(plot)
    character_attributes = character_attributes_from_text(plot)
    character_active_verbs = character_active_verbs_from_text(plot)
    character_patient_verbs = character_patient_verbs_from_text(plot)
    for name in character_names:
        character_list.append(
            {
                'wiki_id': row['wiki_id'],
                'character': name,
                'adj': character_attributes.get(name, []),
                'active': character_active_verbs.get(name, []),
                'patient': character_patient_verbs.get(name, []),
            }
        )

character_df = pd.DataFrame(character_list)
character_df.head()

In [13]:
%%script false --no-raise-error

character_df.to_csv('data/character_attributes.csv')

In [14]:
%%script false --no-raise-error

character_df[character_df['active'].map(len)>5]

## Usage example

In [15]:
from utils.character_attributes_extraction import attributes2vec

In [16]:
characters =  pd.read_csv(
    'data/character_attributes.csv', 
    index_col=0,
    converters={
        "adj": lambda x: x.strip("[]").replace("'","").split(", "), # need this to read list columns from csv
        "active": lambda x: x.strip("[]").replace("'","").split(", "),
        "patient": lambda x: x.strip("[]").replace("'","").split(", ")
        }
    )
characters.head()

,wiki_id,character,adj,active,patient
0,31186339,Snow,[],"[summons, considers]",[]
1,31186339,Haymitch Abernathy,[],[warns],[]
2,31186339,Seneca Crane,[Gamemaker],"[changes, locked]","[summons, convince]"
3,31186339,Rue,[dying],"[draws, cares, draw, trapped]","[hears, stab, comforts, killing]"
4,31186339,Primrose Everdeen,[old],[chosen],[]


In [17]:
for i, r in characters.head().iterrows():
        print(" ".join([str(len(l)) for l in attributes2vec(r)]))

0 2 0
0 1 0
1 2 2
1 4 4
1 1 0
